In [0]:
import pandas as pd
from pyspark.sql.functions import countDistinct,coalesce,col,lit,abs,unix_timestamp

Reading the table into a dataframe

In [0]:
df=spark.read.table('nyc_taxi.bronze.fhv_trips')

Profiling

In [0]:
df.printSchema()

In [0]:
#nothing spotted here

Null Column Check

In [0]:
null_dict={}

for col in df.columns:
  null_dict[col]= df.filter(df[col].isNull()).count()

null_df=pd.DataFrame.from_dict(data=null_dict,orient='index',columns=['null_count'])
null_df.sort_values(by='null_count',ascending=False,inplace=True)

In [0]:
#nulls in pickup and dropoff, can replace with unknown code in zone lookup or drop if it gives no value.
# sr flag is null for non shared rides, no issue
print(null_df)

Duplicate Record Check

In [0]:
%sql
-- duplicates detected
WITH raw_cte AS (
  SELECT * EXCEPT (run_id,_source_file,_ingest_ts,month,year,_rescued_data)
  FROM nyc_taxi.bronze.fhv_trips
)
SELECT
  count(*) AS total_rows,
  count(DISTINCT *) AS distinct_rows_allcols,
  count(DISTINCT to_json(struct(*))) AS distinct_rows_json
FROM raw_cte;


In [0]:
#there is no way of identifying trip uniqueness with certainty. 
# there are rows with empty affiliated bases which is invalid.
dupe_keys= df.groupBy(['dispatching_base_num','Affiliated_base_number','PUlocationID','DOlocationID','pickup_datetime','dropOff_datetime']).count().filter('count > 1')
display(dupe_keys)

Pickup/dropoff date check

In [0]:
# checking for rows where dropoff time < pickup time
date_check_df=df.filter(col('lpep_dropoff_datetime') < col('lpep_pickup_datetime'))
display(date_check_df)


In [0]:
# df.summary was sufficient for spotting outlier and other columns that don't meet business rules